In [7]:
from huggingface_hub import notebook_login
notebook_login()

In [21]:
import numpy as np
import librosa
import soundfile as sf
import json
import torch
import torch.nn.functional as F
from pyannote.audio.pipelines.speaker_verification import PretrainedSpeakerEmbedding
import torchaudio
import os
from tqdm import tqdm
import scipy.signal as signal

In [22]:
with open('embedding.json', 'r') as f:
    target_embedding_data = json.load(f)

target_embedding_np = np.array([target_embedding_data['cpuData'][str(i)] for i in range(512)])

target_embedding_np = target_embedding_np / np.linalg.norm(target_embedding_np)
target_embedding = torch.tensor(target_embedding_np, dtype=torch.float32)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
target_embedding = target_embedding.to(device)

embedding_model = PretrainedSpeakerEmbedding(
    "pyannote/embedding",
    device=device
)

INFO:pytorch_lightning.utilities.migration.utils:Lightning automatically upgraded your loaded checkpoint from v1.2.7 to v2.5.1. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint ../root/.cache/torch/pyannote/models--pyannote--embedding/snapshots/4db4899737a38b2d618bbd74350915aa10293cb2/pytorch_model.bin`


Model was trained with pyannote.audio 0.0.1, yours is 3.3.2. Bad things might happen unless you revert pyannote.audio to 0.x.
Model was trained with torch 1.8.1+cu102, yours is 2.6.0+cu124. Bad things might happen unless you revert torch to 1.x.


In [26]:
def extract_embeddings(waveform_tensor):
    with torch.no_grad():
        embedding = embedding_model(waveform_tensor)

    if isinstance(embedding, np.ndarray):
        embedding = torch.tensor(embedding, device=device)

    embedding_norm = torch.linalg.norm(embedding)
    if embedding_norm > 0:
        embedding = embedding / embedding_norm

    return embedding

def cosine_similarity(a, b):
    if isinstance(a, np.ndarray):
        a = torch.tensor(a, device=device)
    if isinstance(b, np.ndarray):
        b = torch.tensor(b, device=device)
    return F.cosine_similarity(a.flatten().unsqueeze(0), b.flatten().unsqueeze(0))[0]

In [29]:
class WaveformOptimizer:
    def __init__(self, orig_audio_path, sample_rate=16000):
        waveform, sr = librosa.load(orig_audio_path, sr=sample_rate, mono=True)
        self.orig_waveform = torch.tensor(waveform, dtype=torch.float32).unsqueeze(0).to(device)
        self.sample_rate = sample_rate
        self.best_similarity = -1.0
        self.best_waveform = None

    def optimize_using_perturbations(self, num_iterations=500, num_directions=20):
        """Optimize waveform using random perturbations"""
        print(f"Starting optimization with {num_iterations} iterations...")
        current_waveform = self.orig_waveform.clone()
        current_embedding = extract_embeddings(current_waveform)
        current_similarity = cosine_similarity(current_embedding, target_embedding).item()

        print(f"Initial similarity: {current_similarity:.4f}")
        self.best_similarity = current_similarity
        self.best_waveform = current_waveform.clone()

        pbar = tqdm(range(num_iterations))

        max_noise_scale = 0.03
        min_noise_scale = 0.001

        for i in pbar:
            progress = i / num_iterations
            noise_scale = max_noise_scale * (1 - progress) + min_noise_scale * progress

            best_perturbed_similarity = current_similarity
            best_perturbed_waveform = current_waveform.clone()

            for j in range(num_directions):
                noise = torch.randn_like(current_waveform) * noise_scale

                window = torch.hann_window(current_waveform.shape[1], device=device).unsqueeze(0)
                weighted_noise = noise * window

                perturbed_waveform = current_waveform + weighted_noise

                perturbed_embedding = extract_embeddings(perturbed_waveform)
                perturbed_similarity = cosine_similarity(perturbed_embedding, target_embedding).item()

                if perturbed_similarity > best_perturbed_similarity:
                    best_perturbed_similarity = perturbed_similarity
                    best_perturbed_waveform = perturbed_waveform.clone()

            if best_perturbed_similarity > current_similarity:
                current_waveform = best_perturbed_waveform
                current_similarity = best_perturbed_similarity

                if current_similarity > self.best_similarity:
                    self.best_similarity = current_similarity
                    self.best_waveform = current_waveform.clone()

                    pbar.set_description(f"Similarity: {current_similarity:.4f}")

                    if current_similarity > 0.70:
                        print(f"\nEarly stopping at iteration {i+1} with similarity {current_similarity:.4f}")
                        break

        print(f"Finished optimization. Best similarity: {self.best_similarity:.4f}")
        return self.best_waveform, self.best_similarity

    def save_optimized_audio(self, output_path):
        """Save the best waveform to a file"""
        if self.best_waveform is not None:
            waveform_np = self.best_waveform.cpu().numpy()[0]
            sf.write(output_path, waveform_np, self.sample_rate)
            print(f"Saved optimized audio to {output_path}")
            return True
        else:
            print("No optimized waveform available.")
            return False

In [34]:
print("Perturbation-based Optimization")
original_audio = "base_audio.wav"
optimized_audio = "optimized_audio.wav"

optimizer = WaveformOptimizer(original_audio)
_, best_similarity = optimizer.optimize_using_perturbations(
    num_iterations=500,
    num_directions=20
)
optimizer.save_optimized_audio(optimized_audio)

if best_similarity > 0.7:
    print("\nThis solution should successfully authenticate.")
else:
    print("\nSimilarity is below threshold (0.7). Consider re-running the optimization.")

Perturbation-based Optimization
Starting optimization with 500 iterations...
Initial similarity: 0.0365


Similarity: 0.7005:  74%|███████▍  | 370/500 [03:25<01:12,  1.80it/s]


Early stopping at iteration 371 with similarity 0.7005
Finished optimization. Best similarity: 0.7005
Saved optimized audio to optimized_audio.wav

This solution should successfully authenticate.
